In [1]:
import pandas as pd
import sys
import os

# adding project path to sys.path
PROJECT_PATH = os.getcwd().split('Text2Sql')[0] + 'Text2Sql'
sys.path.append(PROJECT_PATH)
# setup path to data
DATA_PATH = os.path.join(PROJECT_PATH, 'Data', 'v3_claude')
# Data/v2_claude/bird_set_stratified
# Data/v2_claude/spider_set_stratified
# Data/v2_claude/spider2_lite_set
BIRD_DATASET_PATH = os.path.join(DATA_PATH, 'bird_set_stratified')
# SPIDER_DATASET_PATH = os.path.join(DATA_PATH, 'spider_set_stratified')
# SPIDER2_LITE_DATASET_PATH = os.path.join(DATA_PATH, 'spider2_lite_set')

OUTPUT_PATH = os.path.join(PROJECT_PATH, 'Data', 'output')

In [2]:
from src.dataloader import DatasetLoader, DatasetInstance

bird_loader = DatasetLoader(BIRD_DATASET_PATH)
# spider_loader = DatasetLoader(SPIDER_DATASET_PATH)
# spider2_lite_loader = DatasetLoader(SPIDER2_LITE_DATASET_PATH) 

bird_instances = bird_loader.load_instances()
# spider_instances = spider_loader.load_instances()
# spider2_lite_instances = spider2_lite_loader.load_instances()

2025-09-28 13:13:01,895 - src.dataloader - INFO - Loading instances from /home/sina/Projects/Thesis/Text2Sql/Data/v3_claude/bird_set_stratified with pattern instance_*.json
2025-09-28 13:13:01,901 - src.dataloader - INFO - Found 227 instance files
2025-09-28 13:13:02,023 - src.dataloader - INFO - Successfully loaded 227 instances


In [3]:
from pipeline.text2sql_enricher import Text2SQLInferencePipeline
# Data/Auth/snowflake_credential.json
import json
with open(os.path.join(PROJECT_PATH, 'Data', 'Auth', 'snowflake_credential.json'), 'r') as f:
    snowflake_config = json.load(f)

# model_config = {
#     "type": "local",
#     "name": "Qwen2.5-Coder-1.5B",
#     "extended_thinking" : True,  
#     "path": os.path.join(PROJECT_PATH,'models','Qwen2.5-Coder-1.5B') # Replace with your actual model path if needed
# }
# Data/Auth/together.ai.api.key/API.key
with open(os.path.join(PROJECT_PATH, 'Data', 'Auth', 'together.ai.api.key', 'API.key'), 'r') as f:
    api_key = f.read().strip()

with open(os.path.join(PROJECT_PATH, 'Data', 'Auth', 'anthropic.api.key', 'text2sql.key'), 'r') as f:
    anthropic_api_key = f.read().strip()

model_config = {
    "type": "together_ai",
    "name": "meta-llama/Llama-3.3-70B-Instruct-Turbo-Free",
    "extended_thinking" : True,  
    "api_key": api_key
}


# model_config = {
#     "type": "anthropic",
#     "name": "claude-sonnet-4-20250514",
#     "extended_thinking" : True,  
#     "api_key": anthropic_api_key
# }
text2sql_pipeline = Text2SQLInferencePipeline(model_config=model_config,
                                              snowflake_config=snowflake_config)

/home/sina/Projects/Thesis/Text2Sql/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-09-28 13:13:16,341 - pipeline.text2sql_enricher - INFO - Initializing Text2SQLInferencePipeline...


In [4]:
def find_instance_by_id(instances : DatasetInstance, id):
    for inst in instances:

        if inst[0].id == id:
            return inst
        
    return None

In [5]:
from src.utils import (extract_sql_query_from_text,
                        check_exact_match,
                        create_sql_prompt,
                        check_execution_accuracy_2
                        )


# sample_instance,sample_instance_file_path = bird_instances[0]  #


sample_instance,sample_instance_file_path = find_instance_by_id(bird_instances, 1404)

question = sample_instance.question
schema = sample_instance.schemas
evidence = sample_instance.evidence
gold_sql = sample_instance.sql

system_message, user_message = create_sql_prompt(question, schema, evidence)

print(f"System Message: {system_message}"
      f"\nUser Message: {user_message}" )

System Message: You are a database expert. You are supposed to provide a SQL query based on the user's question and the provided database schema. Your response must be in JSON format with a field named 'sql' containing the generated SQL query. Example response format: {"sql": "SELECT * FROM table WHERE condition"}
User Message: Identify the type of expenses and their total value approved for 'October Meeting' event. 
 (PS : total value refers to SUM(cost); 'October Meeting' is an event name;)

Here is the database schema:
```
[{'table_name': 'event', 'description': 'The description of columns of the `event` that require more information are as follows:\n\n- event_id (also known as event id) : A unique identifier for the event\n- event_name : event name\n- event_date (also known as event date) : The date the event took place or is scheduled to take place: Where it means that the value is about : [e.g. 2020-03-10T12:00:00]\n- type : The kind of event, such as game, social, election\n- no

In [6]:
raw_response = text2sql_pipeline.model_provider.generate(system_message, user_message)

print(f"Raw Response: {raw_response}")

2025-09-28 13:13:45,275 - httpx - INFO - HTTP Request: POST https://api.together.xyz/v1/chat/completions "HTTP/1.1 200 OK"


Raw Response: {"sql": "SELECT T1.category, SUM(T2.cost) AS total_value FROM budget AS T1 INNER JOIN expense AS T2 ON T1.budget_id = T2.link_to_budget INNER JOIN event AS T3 ON T1.link_to_event = T3.event_id WHERE T3.event_name = 'October Meeting' AND T2.approved = 'true' GROUP BY T1.category"}


In [7]:
generated_sql = extract_sql_query_from_text(raw_response)

print(f"Generated SQL: \n {generated_sql} \n")

print(f"Gold SQL: \n {gold_sql} \n")

Generated SQL: 
 SELECT T1.category, SUM(T2.cost) AS total_value FROM budget AS T1 INNER JOIN expense AS T2 ON T1.budget_id = T2.link_to_budget INNER JOIN event AS T3 ON T1.link_to_event = T3.event_id WHERE T3.event_name = 'October Meeting' AND T2.approved = 'true' GROUP BY T1.category 

Gold SQL: 
 SELECT T1.type, SUM(T3.cost) FROM event AS T1 INNER JOIN budget AS T2 ON T1.event_id = T2.link_to_event INNER JOIN expense AS T3 ON T2.budget_id = T3.link_to_budget WHERE T1.event_name = 'October Meeting' 



In [9]:
db_connection, db_type = text2sql_pipeline.get_db_connection(sample_instance,sample_instance_file_path)

cursor = db_connection.cursor()
cursor.execute(gold_sql)
gold_results = cursor.fetchall()
print(f"Gold Results: \n {gold_results} \n")

the_claude_respond = '''SELECT b.category AS expense_type, SUM(e.cost) AS total_value FROM event ev JOIN budget b ON ev.event_id = b.link_to_event JOIN expense e ON b.budget_id = e.link_to_budget WHERE ev.event_name = 'October Meeting' AND e.approved = 'true' GROUP BY b.category'''

cursor.execute(the_claude_respond)
claude_results = cursor.fetchall()
print(f"Claude Results: \n {claude_results} \n")


cursor.execute(generated_sql)
generated_results = cursor.fetchall()
print(f"Generated Results: \n {generated_results} \n")
cursor.close()

Gold Results: 
 [('Meeting', 175.39)] 

Claude Results: 
 [('Advertisement', 54.25), ('Food', 121.14)] 

Generated Results: 
 [('Advertisement', 54.25), ('Food', 121.14)] 



In [10]:
db_connection, db_type = text2sql_pipeline.get_db_connection(sample_instance,sample_instance_file_path)

# Check execution accuracy
exec_correct, exec_error = check_execution_accuracy_2(
    generated_sql, sample_instance.sql, db_connection
)

# Check exact match
exact_match = check_exact_match(generated_sql, sample_instance.sql)

print(f"Execution Correct: {exec_correct}, Execution Error: {exec_error}")
print(f"Exact Match: {exact_match}")

Execution Correct: False, Execution Error: 
Exact Match: False


In [11]:
evaluation = text2sql_pipeline.evaluate_instance(sample_instance, generated_sql, sample_instance_file_path)

print(f"Evaluation:")
display(evaluation)

2025-09-28 13:14:48,979 - httpx - INFO - HTTP Request: POST https://api.together.xyz/v1/chat/completions "HTTP/1.1 200 OK"


Evaluation:


{'instance': DatasetInstance(id=1404, question="Identify the type of expenses and their total value approved for 'October Meeting' event.", sql="SELECT T1.type, SUM(T3.cost) FROM event AS T1 INNER JOIN budget AS T2 ON T1.event_id = T2.link_to_event INNER JOIN expense AS T3 ON T2.budget_id = T3.link_to_budget WHERE T1.event_name = 'October Meeting'", database={'name': 'student_club', 'path': ['databases/student_club/student_club.sqlite'], 'csv_files': ['bird_set_stratified/databases/student_club/income.csv', 'bird_set_stratified/databases/student_club/budget.csv', 'bird_set_stratified/databases/student_club/zip_code.csv', 'bird_set_stratified/databases/student_club/expense.csv', 'bird_set_stratified/databases/student_club/member.csv', 'bird_set_stratified/databases/student_club/attendance.csv', 'bird_set_stratified/databases/student_club/event.csv', 'bird_set_stratified/databases/student_club/major.csv'], 'type': 'sqlite'}, schemas=[{'table_name': 'event', 'description': 'The descriptio

In [12]:
evaluation['predicted_output']

{'generated_sql': "SELECT T1.category, SUM(T2.cost) AS total_value FROM budget AS T1 INNER JOIN expense AS T2 ON T1.budget_id = T2.link_to_budget INNER JOIN event AS T3 ON T1.link_to_event = T3.event_id WHERE T3.event_name = 'October Meeting' AND T2.approved = 'true' GROUP BY T1.category",
 'execution_correct': False,
 'execution_error': '',
 'exact_match': False,
 'semantic_equivalent': False,
 'semantic_explanation': "The two queries are not semantically equivalent. The Gold SQL Query does not filter expenses by approval status, while the Generated SQL Query only includes approved expenses. Additionally, the Gold SQL Query selects 'type' from the 'event' table, while the Generated SQL Query selects 'category' from the 'budget' table, which may not be the same. The Generated SQL Query also includes a GROUP BY clause, which is necessary because it selects a non-aggregated column 'category', while the Gold SQL Query does not."}

In [13]:
text2sql_pipeline.run_pipeline(instances=bird_instances,
                               save_updated_files=True,
                               output_dir=OUTPUT_PATH + '/Llama-3.3-70B-Instruct-Turbo_bird')

Processing instances:   0%|          | 0/227 [00:00<?, ?instance/s]2025-09-28 13:15:00,467 - pipeline.text2sql_enricher - INFO - Processing instance 1503...
2025-09-28 13:15:11,198 - httpx - INFO - HTTP Request: POST https://api.together.xyz/v1/chat/completions "HTTP/1.1 200 OK"
2025-09-28 13:15:30,405 - httpx - INFO - HTTP Request: POST https://api.together.xyz/v1/chat/completions "HTTP/1.1 200 OK"
2025-09-28 13:15:30,407 - pipeline.text2sql_enricher - INFO - Execution correct: False
2025-09-28 13:15:30,408 - pipeline.text2sql_enricher - INFO - Exact match: False
2025-09-28 13:15:30,408 - pipeline.text2sql_enricher - INFO - Semantic equivalent: True
2025-09-28 13:15:30,412 - pipeline.text2sql_enricher - INFO - --------------------------------------------------
Processing instances:   0%|          | 1/227 [00:29<1:52:47, 29.95s/instance]2025-09-28 13:15:30,413 - pipeline.text2sql_enricher - INFO - Processing instance 691...
2025-09-28 13:15:46,041 - httpx - INFO - HTTP Request: POST ht

{'num_evaluated': 227,
 'num_with_prediction': 227,
 'prediction_rate': 1.0,
 'execution_accuracy': 0.4581497797356828,
 'exact_match_accuracy': 0.10572687224669604,
 'semantic_equivalent_accuracy': 0.6211453744493393,
 'model': {'model_name': 'meta-llama/Llama-3.3-70B-Instruct-Turbo-Free',
  'model_type': 'together_ai',
  'timestamp': '2025-09-28T13:13:16.407279'}}

In [ ]:
# text2sql_pipeline.run_pipeline(instances=spider_instances,
#                                save_updated_files=True,
#                                output_dir=OUTPUT_PATH + '/Llama-3.3-70B-Instruct-Turbo-Free/spider')

Processing instances:   0%|          | 0/250 [00:00<?, ?instance/s]2025-09-20 19:16:19,318 - pipeline.text2sql_enricher - INFO - Processing instance 628...
2025-09-20 19:16:19,814 - httpx - INFO - HTTP Request: POST https://api.together.xyz/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
2025-09-20 19:16:19,815 - openai._base_client - INFO - Retrying request to /chat/completions in 9.000000 seconds
2025-09-20 19:16:29,740 - httpx - INFO - HTTP Request: POST https://api.together.xyz/v1/chat/completions "HTTP/1.1 200 OK"
2025-09-20 19:16:29,749 - pipeline.text2sql_enricher - INFO - Execution correct: False
2025-09-20 19:16:29,749 - pipeline.text2sql_enricher - INFO - Exact match: False
2025-09-20 19:16:29,750 - pipeline.text2sql_enricher - INFO - Semantic equivalent: False
2025-09-20 19:16:29,753 - pipeline.text2sql_enricher - INFO - --------------------------------------------------
Processing instances:   0%|          | 1/250 [00:10<43:18, 10.44s/instance]2025-09-20 19:16:29,755 -

{'num_evaluated': 250,
 'num_with_prediction': 247,
 'prediction_rate': 0.988,
 'execution_accuracy': 0.7327935222672065,
 'exact_match_accuracy': 0.08502024291497975,
 'semantic_equivalent_accuracy': 0.7935222672064778,
 'model': {'model_name': 'meta-llama/Llama-3.3-70B-Instruct-Turbo-Free',
  'model_type': 'together_ai',
  'timestamp': '2025-09-20T15:44:46.799727'}}